In [1]:
from utils import read_from_jsonl
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

In [2]:
#train_data[0]
print(len(train_data))
print(len(test_data))
#test_data = test_data[:3000]

14041
3453


In [8]:
from copy import deepcopy
train_data_encoded = deepcopy(train_data)
test_data_encoded = deepcopy(test_data)

In [9]:
import numpy as np

curr_one_hot_encoding_mapping = {
    1: [1, 0, 0, 0],
    2: [0, 1, 0, 0],
    3: [0, 0, 1, 0],
    4: [0, 0, 0, 1]
}

prev_one_hot_encoding_mapping = {
    # 0 for start of string sequence
    0: [1, 0, 0, 0, 0],
    1: [0, 1, 0, 0, 0],
    2: [0, 0, 1, 0, 0],
    3: [0, 0, 0, 1, 0],
    4: [0, 0, 0, 0, 1]
}

for i, data in enumerate(train_data):
    for j, pos in enumerate(data["pos_tags"]):
        if j == 0:
            prev = prev_one_hot_encoding_mapping[0]
            curr = curr_one_hot_encoding_mapping[pos]
        else:
            prev = prev_one_hot_encoding_mapping[data["pos_tags"][j-1]]
            curr = curr_one_hot_encoding_mapping[pos]
        
        train_data_encoded[i]["pos_tags"][j] = np.array(prev + curr)
        

In [10]:
train_data_encoded[0]


{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': [1, 1, 1, 0, 1, 1, 1, 0, 1],
 'pos_tags': [array([1, 0, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 0, 1]),
  array([0, 0, 0, 0, 1, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 0, 0, 1])]}

In [11]:
for i in train_data_encoded:
    chunk_tags = np.array(i["chunk_tags"])
    i["chunk_tags"] = chunk_tags
    pos_tags = np.array(i["pos_tags"])
    pos_tags.reshape(1, pos_tags.shape[0], pos_tags.shape[1])
    i["pos_tags"] = pos_tags


In [12]:
train_data_encoded[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'chunk_tags': array([1, 1, 1, 0, 1, 1, 1, 0, 1]),
 'pos_tags': array([[1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1]])}

In [13]:
for i in range(2):
    print(train_data_encoded[i]["pos_tags"].shape)
    print(train_data_encoded[i]["chunk_tags"].shape)

(9, 9)
(9,)
(2, 9)
(2,)


## Dataloader implementing Above things is in utils.py

In [3]:
import torch
from utils import read_from_jsonl
from utils import DataLoader
import matplotlib.pyplot as plt
train_data = read_from_jsonl("train.jsonl")
test_data = read_from_jsonl("test.jsonl")

train_data_loader = DataLoader(train_data[:3000], batch_size=len(train_data[:3000]))
test_data_loader = DataLoader(test_data, batch_size=1)

In [4]:
len(train_data), len(train_data_loader) 

(14041, 3000)

In [5]:
from utils import RecurrentPerceptron

a = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)
#a.load_model('model.pkl')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4497814476490021
Epoch 2 : Avg Loss : 0.43987584114074707
Epoch 4 : Avg Loss : 0.4397176206111908
Epoch 6 : Avg Loss : 0.43966320157051086
Epoch 8 : Avg Loss : 0.4396343231201172
Epoch 10 : Avg Loss : 0.43962419033050537


In [6]:
a.check_conditions()

(tensor(16), 16)

In [7]:
acc, prec, rec, f1 = a.test_from_dataloader(test_data_loader, verbose=True)

  0%|          | 0/3453 [00:00<?, ?it/s]

In [8]:
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8434
Precision: 0.8100
Recall: 0.9925
F1 Score: 0.8920


In [82]:

# running this will overwrite the model file
a.save_model(path='/Users/keshav/Study/CS772/natural-language-processing-main 2/assignment-2', name='model_3000_10.pkl')

File already exists, overwriting


In [83]:
# final weights
a.W, a.V

(tensor([ 6.6279, -1.8724, -3.7012, -2.8595,  1.8169,  0.4002,  4.3571,  0.7107,
          5.5893]),
 tensor(-0.9459))

In [9]:
def k_fold_cross_validation(model, data, k=5, nepochs=10, batch_size=1):


    np.random.shuffle(data)

    fold_size = len(data) // k
    folds = [data[i:i+fold_size] for i in range(0, len(data), fold_size)]

    accuracies = []
    precisions = []
    recalls = []
    f1s = []

    for i in range(k):
        validation_data = folds[i]
        train_data = [d for j, fold in enumerate(folds) if j != i for d in fold]
        #print(len(train_data))
        #print(len())

        train_loader = DataLoader(train_data, batch_size=len(train_data))
        validation_loader = DataLoader(validation_data, batch_size=batch_size)

        model[i].train_from_dataloader(train_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)
        print(model[i].check_conditions())

        accuracy, prec, rec, f1 = model[i].test_from_dataloader(validation_loader, verbose=False)
        accuracies.append(accuracy)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)

        print(f"Fold {i+1}/{k} - Validation Accuracy: {accuracy:.4f}")
        print(f"Fold {i+1}/{k} - Validation Precision: {prec:.4f}")
        print(f"Fold {i+1}/{k} - Validation Recall: {rec:.4f}")
        print(f"Fold {i+1}/{k} - Validation F1: {f1:.4f}")

    avg_accuracy = np.mean(accuracies)
    avg_precisions = np.mean(accuracies)
    avg_recalls = np.mean(recalls)
    avg_f1s = np.mean(f1s)

    print(f"\nAverage Validation Accuracy: {avg_accuracy:.4f}")
    print(f"\nAverage Validation Precision: {avg_precisions:.4f}")
    print(f"\nAverage Validation Recall: {avg_recalls:.4f}")
    print(f"\nAverage Validation F1: {avg_f1s:.4f}")

    return avg_accuracy, avg_precisions, avg_recalls, avg_f1s


In [10]:
import numpy as np
train_data = read_from_jsonl("train.jsonl")
train_data = train_data[:3000]

# Initialize the model
a1 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a2 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a3 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a4 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
a5 = RecurrentPerceptron(clip_grads=True, clip=1., verbose=False)
model = [a1,a2,a3,a4,a5]
#a.train_from_dataloader(train_data_loader, lr=0.05, nepochs=10, debug=False, print_interval=2)

# Perform 5-fold cross-validation
k_fold_cross_validation(model, train_data, k=5, nepochs=10, batch_size=1)

#print(f"\nOverall 5-Fold Cross-Validation Accuracy: {avg_accuracy:.4f}")

2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.49011749029159546
Epoch 2 : Avg Loss : 0.4841000735759735
Epoch 4 : Avg Loss : 0.4838743209838867
Epoch 6 : Avg Loss : 0.48380523920059204
Epoch 8 : Avg Loss : 0.483770489692688
Epoch 10 : Avg Loss : 0.4837600290775299
(tensor(16), 16)
Fold 1/5 - Validation Accuracy: 0.8886
Fold 1/5 - Validation Precision: 0.8633
Fold 1/5 - Validation Recall: 0.9947
Fold 1/5 - Validation F1: 0.9244
2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.479560911655426
Epoch 2 : Avg Loss : 0.47662532329559326
Epoch 4 : Avg Loss : 0.476403146982193
Epoch 6 : Avg Loss : 0.4763326644897461
Epoch 8 : Avg Loss : 0.4762992858886719
Epoch 10 : Avg Loss : 0.47628864645957947
(tensor(16), 16)
Fold 2/5 - Validation Accuracy: 0.8911
Fold 2/5 - Validation Precision: 0.8685
Fold 2/5 - Validation Recall: 0.9937
Fold 2/5 - Validation F1: 0.9269
2400


...

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.5019621849060059
Epoch 2 : Avg Loss : 0.4893997609615326
Epoch 4 : Avg Loss : 0.4891539514064789
Epoch 6 : Avg Loss : 0.4890761375427246
Epoch 8 : Avg Loss : 0.4890381395816803
Epoch 10 : Avg Loss : 0.48902612924575806
(tensor(16), 16)
Fold 4/5 - Validation Accuracy: 0.8965
Fold 4/5 - Validation Precision: 0.8753
Fold 4/5 - Validation Recall: 0.9931
Fold 4/5 - Validation F1: 0.9305
2400


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 : Avg Loss : 0.4947036802768707
Epoch 2 : Avg Loss : 0.4854482114315033
Epoch 4 : Avg Loss : 0.48522648215293884
Epoch 6 : Avg Loss : 0.485160768032074
Epoch 8 : Avg Loss : 0.4851287305355072
Epoch 10 : Avg Loss : 0.4851182997226715
(tensor(16), 16)
Fold 5/5 - Validation Accuracy: 0.8968
Fold 5/5 - Validation Precision: 0.8750
Fold 5/5 - Validation Recall: 0.9924
Fold 5/5 - Validation F1: 0.9300

Average Validation Accuracy: 0.8942

Average Validation Precision: 0.8942

Average Validation Recall: 0.9933

Average Validation F1: 0.9286


(0.8941654249100065,
 0.8941654249100065,
 0.9933083322404525,
 0.9286485825865753)

In [75]:
print(f"\nOverall 5-Fold Cross-Validation Accuracy: {avg_accuracy:.4f}")


Overall 5-Fold Cross-Validation Accuracy: 0.8941
